In [ ]:
%load_ext autoreload
%autoreload 2

import os
ipy = get_ipython()
#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
from os.path import join as pjoin
import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
import error_sensitivity
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
import pandas as pd

#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
#scripts_dir = pjoin(data_dir_general,'full_experiments','scripts2')

data_dir_general = os.path.expandvars('$DATA_QUENTIN')
data_dir_input = os.path.expandvars('$DATA_MEMORY_ERRORS_STAB_AND_STOCH')
#scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'previous_analyses')
scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'))


import gc

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
print('seaborn ver = ', sns.__version__)
#assert sns.__version__ == '0.12.2', sns.__version__
import matplotlib as mpl

In [ ]:
def setPlotColors():
    COLOR = 'black'
    mpl.rcParams['text.color'] = COLOR
    mpl.rcParams['axes.labelcolor'] = COLOR
    mpl.rcParams['xtick.color'] = COLOR
    mpl.rcParams['ytick.color'] = COLOR


    sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white',
               'grid.color':'grey', 'grid.alpha':0.5})
    global tgtc
    tgtc = ['b','g','r','brown']
setPlotColors()

In [ ]:
d = '/home/demitau/data_Quentin/pilot_studies/context_change_behav/'
#fnb = 'dima_test_context_change_20230309_164851'
#fnb = '2023-SE1-pilot2_context_change_20230310_161928'
fnb = '2023-SE1-017_context_change_20230425_161049'
fn = fnb + '.log'
fnp = fnb + '.param'

# Main

In [ ]:
import gc; gc.collect()

In [ ]:
from datetime import datetime

In [ ]:
from behav_proc import readParamFiles, addBasicInfo, getGeomInfo
fnp = fnb + '.param'
params, phase2trigger, trigger2phase, stage2pars  = readParamFiles(fnp, d)
home_position, target_coords = getGeomInfo(params)
hitr = float(params['radius_target']) + float(params['radius_cursor']) / 2

load = True
#load = True
if load:
    #dfallphs = pd.read_pickle(pjoin(d,'row=frame_allph.pkl.zip'))
    dfc_all      = pd.read_pickle(pjoin(d,'row=frame.pkl.zip'))        
    dfcpc_all    = pd.read_pickle(pjoin(d,'row=pause.pkl.zip'))
    dfctc_all    = pd.read_pickle(pjoin(d,'row=target_chagne.pkl.zip'))
    fnf_dfcc = pjoin(d,'row=trial.pkl.zip')
    dfcc_all     = pd.read_pickle(fnf_dfcc)
    
    dt = datetime.fromtimestamp(os.stat(fnf_dfcc).st_mtime )
    print('date dfcc = ',dt)
    
    fnf = pjoin(d,'es_row=trial.pkl.zip')
    dt = datetime.fromtimestamp(os.stat(fnf).st_mtime )    
    print('date es = ',dt)
    

    df_all_multi_tsz_orig = pd.read_pickle(fnf)

In [ ]:
for df_ in [dfcc_all    ,dfc_all     ,dfcpc_all   ,dfctc_all]:
    assert np.sum( df_.duplicated()) == 0
    
assert dfcc_all.index.duplicated().sum() == 0

In [ ]:
df_all_multi_tsz = df_all_multi_tsz_orig

In [ ]:
mx = df_all_multi_tsz.groupby(['subject','trial_index']).size().max()
print('Duplication for ES = ',mx)

In [ ]:
# info about ES dataset
cols_undup = ['subject', 'trial_index', 'error_data_for_calc',
       'trial_shift_size', 'trial_group_col_calc', 'time_locked']
assert not df_all_multi_tsz.duplicated(cols_undup).any()
for col in cols_undup:
    if col == 'trial_index':
        continue
    print(col, df_all_multi_tsz[col].unique())

In [ ]:
subjects = dfcc_all['subject'].unique()
print(subjects)

subj = subjects[3]
print(subj)
dfccos  = dfcc_all.query('subject == @subj')
dfcos   = dfc_all.query('subject == @subj')
dfcpcos = dfcpc_all.query('subject == @subj')
#dfcc = dfccos

In [ ]:
ynames = ['error_endpoint_ang', 'error_lh_ang', 'error_unpert_lh_ang', 'error_area_ofb_signed', 
      'error_area_signed', 'error_area_signed_scaled_ed']
#dfcclline[ynames].plot(x='trial_index')
for yname in ynames:
    plt.figure(figsize=(12,3))
    ax = sns.lineplot(dfccos, x='trial_index', y= yname)
    ax.axhline(y=0, c='r')

In [ ]:
colns_nonhit = ['nonhit', 'error_area_signed_nonhit',
       'error_area_signed_scaled_ed_nonhit', 'error_lh_ang_nonhit']

## Process err sens

In [ ]:
gc.collect()

In [ ]:
#df_all_multi_tsz = pd.read_pickle(pjoin(d,'es_row=trial.pkl.zip'))
coln_pairs = [ ('error_lh_ang','error_unpert_lh_ang','error_lh_ang_nonhit','lh_ang'),
        ('error_area2_signed_nn_scaled_ed', None,
               'error_area2_signed_nn_scaled_ed_nonhit',
                'signed_area2_nn_scaled_ed') ] 


assert set( df_all_multi_tsz['error_data_for_calc'] ) == set(  [cp[-1] for cp in coln_pairs])

In [ ]:
from behav_proc import truncateDf
coln = 'err_sens'
dfc_multi_tsz = truncateDf(df_all_multi_tsz, coln, q=0.05, infnan_handling='discard',
                          cols_uniqify = ['trial_shift_size','trial_group_col_calc','error_data_for_calc'])
dfc_multi_tsz = dfc_multi_tsz.query('dist_rad_from_prevtgt2 != "nan"')

#df_all_multi_tsz.to_pickle(pjoin(d,'es_trunc_row=trial.pkl.zip'),compression='zip')

In [ ]:
dfes = df_all_multi_tsz.query('error_data_for_calc == "lh_ang"')
from behav_proc import thr_lh_ang_rad as thr_ang
inds1 = dfes.query('(prev_error >= @thr_ang) or (prev_error <= -@thr_ang)' ).index

dfes = df_all_multi_tsz.query('error_data_for_calc == "signed_area2_nn_scaled_ed"')
from behav_proc import thr_signed_area2_nn_scaled_ed as thr_area
inds2 = dfes.query('(prev_error >= @thr_area) or (prev_error <= -@thr_area)' ).index

In [ ]:
len(df_all_multi_tsz)

In [ ]:
from behav_proc import truncateDf
dfc_multi_tsz = df_all_multi_tsz.loc[inds1.append(inds2)]
dfc_multi_tsz = truncateDf(df_all_multi_tsz, 'err_sens', q=None, infnan_handling='discard',
        cols_uniqify = cols_undup[2:])

In [ ]:
100 * len(dfc_multi_tsz) / len(df_all_multi_tsz)

In [ ]:
gc.collect()

### basic plots

In [ ]:
for cn in calc_names:
    dfc_ = dfc_multi_tsz.query('trial_shift_size == 1 and error_data_for_calc == @cn')
    plt.figure(figsize=(12,3))
    #dfc_ = df_all_multi_tsz.query('trial_shift_size == 1')
    ax = sns.lineplot(x="trials", y='err_sens',data=dfc_,  estimator='mean', 
                      errorbar='se',
                     hue = 'trial_group_col_calc')#, hue='block')
    ax.axhline(y=0, c='red', ls=':')
    ax.legend(loc='lower right')
    ax.set_title(cn)

In [ ]:
dfone = df_all.query('subject == @subjects_[0]')
yname = coln_pairs[0][0]

ebarg = ''
ax = sns.lineplot(x="trials", y=yname,data=dfone,  estimator='mean', errorbar='se')#, hue='block')
ax.axhline(0,ls=':',c='red')
ax.plot(dfone['trials'].to_numpy(), dfone['perturbation'].to_numpy(), ls=':',c='y')


##### violin plots

In [ ]:
fg = sns.catplot(kind='violin', data=dfcc_all,
                 y='error_area2_signed_nn_scaled_ed',                                   
                 x='perturbation', hue='trial_type', 
                 col = 'special_block_type',
                 figsize=(4,3))
for ax in fg.axes.flatten():
    #ax.set_ylim(-1e5,1e5)
    ax.axhline(0,c='r',ls=':')

In [ ]:
fg = sns.catplot(kind='violin', data=dfcc_all,
                 y='error_lh_ang',                                   
                 x='perturbation', hue='trial_type', 
                 col = 'special_block_type',
                 figsize=(4,3))
for ax in fg.axes.flatten():
    ax.set_ylim(-100,100)
    ax.axhline(0,c='r',ls=':')

In [ ]:
#dfc_.groupby(['subject', 'trial_index']).size()

In [ ]:
#for cn in calc_names:
for df_to_plot in [dfc_multi_tsz]: #, df_all_multi_tsz]:
    dfc_ = df_to_plot.query('trial_shift_size == 1 and trial_group_col_calc == "trials" and '
                            ' trial_type != "error_clamp" and '
                           ' special_block_type != "error_clamp_sandwich"')  
    # error_data_for_calc = @cn
    assert dfc_.groupby(['subject', 'trial_index']).size().max() == 3
    print(len(dfc_))
    #plt.figure(figsize=(6,3))
    #dfc_ = df_all_multi_tsz.query('trial_shift_size == 1')
    fg = sns.catplot(kind='violin', data=dfc_,
                     y='err_sens', col = 'error_data_for_calc',                                   
                     x='perturbation', figsize=(6,3))
    #x = 'dist_rad_from_prevtgt2',
    for ax in fg.axes.flatten():
        ax.set_ylim(-7,7)
        ax.axhline(0,c='r',ls=':')
        #ax.set_title(cn)

In [ ]:
#for cn in calc_names:
for df_to_plot in [dfc_multi_tsz]: #, df_all_multi_tsz]:
    dfc_ = df_to_plot.query('trial_shift_size == 1 and trial_group_col_calc == "trials" and '
                            ' trial_type != "error_clamp" and '
                           ' special_block_type != "error_clamp_sandwich"')  
    # error_data_for_calc = @cn
    assert dfc_.groupby(['subject', 'trial_index']).size().max() == 3
    print(len(dfc_))
    #plt.figure(figsize=(6,3))
    #dfc_ = df_all_multi_tsz.query('trial_shift_size == 1')
    fg = sns.catplot(kind='violin', data=dfc_,
                     y='err_sens', col = 'error_data_for_calc',                                   
                     x='perturbation', row='subject', figsize=(6,3))
    #x = 'dist_rad_from_prevtgt2',
    for ax in fg.axes.flatten():
        ax.set_ylim(-7,7)
        ax.axhline(0,c='r',ls=':')
        #ax.set_title(cn)

In [ ]:
#for cn in calc_names:
for df_to_plot in [dfc_multi_tsz]: #, df_all_multi_tsz]:
    dfc_ = df_to_plot.query('trial_shift_size == 1 and trial_group_col_calc == "trials" and '
                            ' trial_type != "error_clamp" and '
                           ' special_block_type != "error_clamp_sandwich"')  
    # error_data_for_calc = @cn
    assert dfc_.groupby(['subject', 'trial_index']).size().max() == 3
    print(len(dfc_))
    #plt.figure(figsize=(6,3))
    #dfc_ = df_all_multi_tsz.query('trial_shift_size == 1')
    fg = sns.catplot(kind='violin', data=dfc_,
                     y='err_sens', col = 'error_data_for_calc',                                   
                     x='perturbation', row='tgti_to_show', figsize=(6,3))
    #x = 'dist_rad_from_prevtgt2',
    for ax in fg.axes.flatten():
        ax.set_ylim(-7,7)
        ax.axhline(0,c='r',ls=':')
        #ax.set_title(cn)

In [ ]:
# all subj separately
subj = subjects[0]
df_to_plot = df_all_multi_tsz

ynames_add = ['error_endpoint_ang', 'jax2']
ynames_every = ['err_sens', 'correction']

titest = 18
ww = 12; hh = 2
nr = len(calc_names) * (len(ynames_every) + 2) + len(ynames_add);
nc = 1
fig, axs = plt.subplots(nr,nc, figsize=(nc*ww,nr*hh))
axs = axs.reshape((nr,nc))
def fplot(ax, dfc_, yn):
    ax = sns.lineplot(x="trials", y=yn, data=dfc_,  estimator='mean', 
                      errorbar='se',hue = 'trial_group_col_calc', ax=ax)#, hue='block')
    ax.axhline(y=0, c='red', ls=':')
    ax.legend(loc='lower right')

axi = 0
for cni, cn in enumerate(calc_names):
    #df_all_multi_tsz
    #df_to_plot = dfc_multi_tsz    
    dfc_ = df_to_plot.query('trial_shift_size == 1 and trial_group_col_calc == "trials" '
            'and error_data_for_calc == @cn and trial_index < @titest and subject == @subj')        
    for yn in ynames_every:
        ax = axs[axi, 0 ]
        fplot(ax, dfc_, yn)
        ax.set_ylabel(f'{yn}: {cn}')
        ax.set_title(f'{yn}: {cn}')
        axi += 1
        
    for ii in range(2):
        ax = axs[axi, 0 ]
        yn = coln_pairs[cni][ii]
        fplot(ax, dfc_, yn)
        ax.set_title(f'{yn}: {cn}')
        axi += 1
        
    
for yn in ynames_add:
    ax = axs[axi, 0]
    fplot(ax, dfc_, yn)
    axi += 1

for ax in axs.flatten():
    ax.set_xlim(0,titest)
    #dfc_ = df_all_multi_tsz.query('trial_shift_size == 1')
    
plt.tight_layout()


In [ ]:
import gc; gc.collect()

##### REST and  move prep anomalise

In [ ]:
tis_cur = tis[10:30]
tis_cur = tis[10:11]
dfr = dfc_all.query('subject == @subjects[0] and trial_index in @tis_cur')

In [ ]:
dfr = dfr.sort_values('time')

In [ ]:
dfr.shift

In [ ]:
dfccos['time_mvt'] = -100

In [ ]:
#dfccos = dfccos.copy()

In [ ]:
inds

In [ ]:
dfccos.query()

In [ ]:
#dfccos_hit = dfccos[~dfccos['nonhit'] ]
#for ti in tis:


In [ ]:
dfccos['time_mvt']

In [ ]:
#nfs.describe()

In [ ]:
nf1  = float( params['time_at_home'] ) * float(params['FPS'] )
nf2 = float( params['motor_prep_duration'] ) * float(params['FPS'] )
print(nf1,nf2)

In [ ]:
triggs

In [ ]:
nf1 * 1.5

In [ ]:
subj2nt = df_unproc[['subject','trial_index']].groupby('subject').nunique()
subj2nt = subj2nt.to_dict('index')
#{'2023-SE1-001': {'trial_index': 637},
# '2023-SE1-002': {'trial_index': 636},

In [ ]:
subj2nt

In [ ]:
subj2qv

In [ ]:
subj2qv

In [ ]:
for phase in ['REST', 'GO_CUE_WAIT_AND_SHOW']:
    print(phase)
    phases = [phase]
    triggs = [phase2trigger[ph] for ph in phases]
    df0 = df_unproc.query('current_phase_trigger in @triggs')
    
    nfs = df0.groupby(['subject','trial_index']).size().\
        reset_index().rename(columns={0:'numframes'})
    
    
    dftmp = nfs.groupby('subject')['numframes'].quantile(0.75).reset_index().set_index('subject')
    subj2qv = dftmp.to_dict('index')
    print(subj2qv)
    
    def f(row):
        qv = subj2qv[row['subject']]['numframes']
        return row['numframes'] > qv
    nfs['bad'] = nfs.apply(f,1)
    nfs
    
    display( nfs.describe() )
    
    #qv = nfs['numframes'].quantile(0.5)
    #qv = nf1 * 1.5
    nfs_bad = nfs[nfs['bad']]
    
    grp = nfs_bad[['subject','trial_index']].groupby('subject')
    display(grp.size())
    
    plt.figure()
    sns.lineplot(nfs, x='trial_index', y ='numframes')

In [ ]:
for phase in ['REST', 'GO_CUE_WAIT_AND_SHOW'][1:]:
    print(phase)
    phases = [phase]
    triggs = [phase2trigger[ph] for ph in phases]
    df0 = df_unproc.query('current_phase_trigger in @triggs')
    
    nfs = df0.groupby(['subject','trial_index']).size().\
        reset_index().rename(columns={0:'numframes'})
    sns.lineplot(nfs.query('subject == @subjects[0]'), x='trial_index', y ='numframes')

In [ ]:
#nfs_bad.query('sub')


##### plot time resolved with pert and blocks

In [ ]:
(dfc_multi_tsz['trial_type'] == 'error_clamp').sum()

In [ ]:
from config2 import path_fig

In [ ]:
#subj = subjects[1]
dfr_all = []
for subj in subjects:

    #df_to_plot = df_all_multi_tsz
    df_to_plot = dfc_multi_tsz

    #' and special_block_type != "error_clamp_sandwich"'
    qs0 = ('trial_shift_size == 1 and trial_group_col_calc == "trials"'
           ' and trial_type != "error_clamp0"'                      
                f' and subject == "{subj}"')
    #dfc0_ = df_to_plot.query(qs0) 
    r = []
    for cni, cn in enumerate(calc_names):
        #df_all_multi_tsz
        #df_to_plot = dfc_multi_tsz            
        qs = qs0 + f' and error_data_for_calc == "{cn}" '
        dfc_ = df_to_plot.query(qs)

        print(len(dfc_))

        dfc_ = dfc_.set_index('trials')
        if cni != 0:
            dfc_ = dfc_[['err_sens','correction']]
        dfc_ = dfc_.rename(columns={'err_sens': f'err_sens:{cn}', 'correction': f'correction:{cn}'})
        r += [ dfc_]

    #[dfc0_.drop(labels=['err_sens','correction'], axis=1)]
    dfr0 = pd.concat( r, axis=1)
    print(len(dfr0))

    dfr0 = dfr0.reset_index()
    
    print( 'num nans = ', dfr0['trial_index'].isna().sum() )
    dfr = dfr0[~dfr0['trial_index'].isna() ] 

    dfr_all += [dfr]
    #dfr['block_ind']
    #dfr.query('trial_type != "error_clamp"').groupby('block_ind')
    #coln_pairs

    ynames = []
    for yn1, yn2, nh, cn in coln_pairs:
        ynames += [f'err_sens:{cn}', f'correction:{cn}', yn1, yn2]
    ynames += ['error_endpoint_ang']
    print(ynames)

    #assert len(dfc_) == len(dfr), (len(dfc_), len(dfr))
    #dfr[['trial_index','err_sens:signed_area','err_sens:lh_ang']]
    dfcpcos_, dfcos_ = dfcpc_all.query('subject == @subj'), dfc_all.query('subject == @subj')

    assert 0 == sum( dfr['trial_index'].duplicated() )

    xlim = 0,150
    # dfc_ = dfr.query('trial_shift_size == 1 and trial_group_col_calc == "trials" '
    #         'and error_data_for_calc == @cn and trial_index < @xlim[1] and subject == @subj')
    dfr_ = dfr.query('trial_index < @xlim[1]')

    from behav_proc import multiplot
    #['err_sens', 'correction']
    multiplot(dfr_, dfcpcos_, dfcos_, ynames, start_ind=0, xlim = xlim , ww=14, hh=1.5,
             show_titles = True)
    plt.tight_layout()
    #plt.savefig( pjoin(path_fig, f'{subj}_{",".join(ynames)}.pdf' ) )
    plt.savefig( pjoin(path_fig, f'{subj}_nr={len(ynames)}_xlim={xlim}.pdf' ) )
    plt.close()
    
dfr_all = pd.concat(dfr_all).reset_index()

In [ ]:
dfr['trial_index'].isna().sum()

In [ ]:
print( dfr['trial_index'].isna().sum() )
print( dfr_all['trial_index'].isna().sum() )
dfr_all = dfr_all[~dfr_all['trial_index'].isna() ] 

In [ ]:
#dfr_all['subject']

In [ ]:
subjects = list(sorted(subjects))

In [ ]:
subjects_subset

In [ ]:
sns.lineplot(dfcc, x='trial_index', y ='feedback_abs')

In [ ]:
sns.lineplot(dfcc, x='trial_index', y ='feedback')

In [ ]:
dfcc['error_intdist']

In [ ]:
dfcc.columns

# error plots

In [ ]:
# TODO maybe I should set error to nan for pause trials

In [ ]:
#pause_inds = dfcpcos['trial_index'].to_numpy()
pause_inds = dfcpcos['trial_index'].unique()
block_ends = dfcos.groupby(['block_ind'])['trial_index'].max()

In [ ]:
block_starts = markBlockStarts(dfes, ax, dict(ymin = -1, ymax = 1, color='grey', ls=':') )

In [ ]:
# coef = dfcc['error_area_signed'].quantile(0.75) / 15
# dfcc['error_aug3'] = dfcc['error_area_signed'] + coef * dfcc['error_pert_adj']


In [ ]:
# not needed because pauses are ALRREADY NOT in dfcc

# inds = dfccos['trial_index'].isin(pause_inds)
# for yname in ['error_aug2', 'error_intdist2', 'error_intdist2nn','error_intdist','error_distance',              
#              'error_distance_lh']:
#     dfccos.loc[inds, yname] = np.nan

In [ ]:
# c = dfccos['error'] > 180
# dfccos.loc[c, 'error'] =  dfccos.loc[c, 'error'] - 360
# c = dfccos['error'] < -180
# dfccos.loc[c, 'error'] =  dfccos.loc[c, 'error'] + 360
# c = dfccos['perturbation'] < 0
# dfccos['error_pert_adj'] = dfccos['error']
# dfccos.loc[c, 'error_pert_adj'] = -dfccos.loc[c,'error_pert_adj']


In [ ]:
dfccos  = dfcc_all.query('subject == @subj').copy()

In [ ]:
numtrain = params['num_training']

In [ ]:
from behav_proc import markBlockStarts

In [ ]:
dfccos['error_lh_ang_deg'] = dfccos['error_lh_ang'] * 180 / np.pi

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
#xlim = None
xlim = 0,200
grp = dfccos.query('trial_type != "error_clamp"').groupby('block_ind')
block_ends = grp['trial_index'].max()
df_ = dfccos.query('trial_index >= @numtrain')

# ynames = ['error','error_pert_adj', 'error_distance',
#           'error_area_signed_nn',
#           'error_area_signed', 'error_aug3', 
#           'time_lh', 'traj_length']
ynames = ['time_lh', 'traj_length_adj',
          'error_area2_signed_nn',          
           'error_lh_ang_deg']
ww = 20; hh=3
nr = len(ynames)
fig,axs =  plt.subplots(nr,1,figsize=(ww, nr*hh))
axs = axs.flatten()
for axi, yname in enumerate(ynames):    
    ax = axs[axi]    
    sns.lineplot(df_, x='trial_index', y =yname, ax=ax)    
    from config2 import path_fig
    
    mult = df_[yname].max() / 40                
    ax.plot(dfccos['trial_index'], dfccos['perturbation']._values * mult,
             alpha=0.4, c='r', label='pert')
    ax.vlines(pause_inds, -20 * mult, 20 * mult, 
              color='green', ls = ':', lw=3,
             label='pause')
    
#     ax.vlines(block_ends, -20 * mult, 20 * mult, 
#  color='grey', ls = '--',
#              label='block')
    block_starts = markBlockStarts(df_, ax, 
        dict(ymin = -20 * mult, ymax =  20 * mult, 
             color='grey', ls='--', alpha=0.5) )
    
    ax.vlines(ECs, 0.5 * 20 * mult, 20*mult, 
              color='magenta',lw=2, ls='--')

    ax.plot(dfccos['trial_index'], dfccos['tgti_to_show'] * 7 * mult, 
            color='cyan', alpha=0.5,
           label='target', ls=':')
    ax.grid(False)
    ax.axhline(y=0, ls=':', c='red')

    if axi == nr - 1:
        ax.legend(loc='lower right')
    ax.set_xlim(*xlim)
plt.tight_layout()
plt.savefig( pjoin(path_fig, f'{subj}_{",".join(ynames)}.pdf' ) )

In [ ]:
from nf_poster_plots import *
locs = oneSubjErrDyn(dfccos, pause_inds, subj=subj)
#axs= locs['axs']
plt.close()

In [ ]:
dfcc

In [ ]:
#dfcc['error_distance']

In [ ]:
# TODO: error clamp need to be handled differently?

In [ ]:
#sns.lineplot(dfcc, x='trial_index', y ='error_eucl')
#sns.lineplot(dfcc, x='trial_index', y ='feedbackX')
sns.lineplot(dfcc, x='trial_index', y ='feedback_abs',label='feedback_abs')
sns.lineplot(dfcc, x='trial_index', y ='org_feedback_abs',label='org_feedback_abs',
            ls='--')
#sns.lineplot(dfcc, x='trial_index', y ='feedback')
#sns.lineplot(dfcc, x='trial_index', y ='error')
sns.lineplot(dfcc, x='trial_index', y ='perturbation', alpha=0.4, c='r',label='pert')
sns.lineplot(dfcc, x='trial_index', y ='target_locs', c='g',label='tgt')
plt.gca().vlines(dfcpc['trial_index'], -20,10, color= 'cyan', ls='--')
plt.gca().vlines(dfctc['trial_index'], -20,-17, color= 'gray', ls='--',alpha=0.4)
plt.legend()

# summary one remnants

In [ ]:
pause_inds = np.array(pause_inds)
dfccos['pause_rel'] = -100
dfccos.loc[dfccos['trial_index'].isin(pause_inds-1), 'pause_rel'] = -1
dfccos.loc[dfccos['trial_index'].isin(pause_inds+1), 'pause_rel'] = 1
# for pi in pause_inds:
#     piprev=  pi -1
#     pinext=  pi +1
#     df_ = dfccos.query('trial_index == @piprev')


In [ ]:
#df_['pause_rel']

In [ ]:
num_coupled_EC = \
    np.sum( dfccos.query("trial_type == 'error_clamp'")['trial_index'].diff() == 1 )
print(f'num_coupled_EC = {num_coupled_EC}')

In [ ]:
len(block_starts), len(block_ends)

In [ ]:


block_starts = grp['trial_index'].min()
block_starts = block_starts.to_dict()

block_ends = grp['trial_index'].max()
block_ends = block_ends.to_dict()


block_bounds = {}
for bi in block_starts.keys():
    block_bounds[bi] = block_starts[bi],block_ends[bi]
block_bounds

## Start of block vs end of block

In [ ]:
dfccos.columns

In [ ]:
dfccos['trial_type'].unique()

In [ ]:
grp = dfccos.query('trial_type != "error_clamp"').groupby('block_ind')

df1 = grp['trial_index'].min().to_frame().rename(columns={'trial_index':'block_start_trial_index'})
df2 = grp['trial_index'].max().to_frame().rename(columns={'trial_index':'block_end_trial_index'})
block_bounds = pd.concat( [df1, df2], axis=1)

sts = block_bounds.query('block_ind >= 0')['block_start_trial_index']._values
df_block_start = dfccos.loc[dfccos['trial_index'].isin(sts)]

ends = block_bounds.query('block_ind >= 0')['block_end_trial_index']._values
df_block_end = dfccos.loc[dfccos['trial_index'].isin(ends)]

ynames = ['error_pert_adj', 'error_area_signed']
#'error', 


df_block_start = df_block_start[['block_ind', 'trial_type'] + ynames].set_index('block_ind')
#ynames_mod = [yn + '_start' for yn in ynames]
#rd = dict(zip(ynames,ynames_mod))
#df_block_start = df_block_start.rename(columns=rd)
df_block_start['type'] = 'start'

df_block_end =   df_block_end[['block_ind', 'trial_type'] + ynames].set_index('block_ind')
df_block_end['type'] = 'end'

df_startend = pd.concat([df_block_start, df_block_end])

In [ ]:
df_startend

In [ ]:
for yname in ynames:
    plt.figure()
    sns.violinplot(df_startend, y=yname, x='type', hue='trial_type', split=True)

In [ ]:
df_ = dfccos.query('pause_rel > -10')
#df_ = df_.query('error < 150')

ynames = ['error_area_signed',  'error_distance', 'error']
for yname in ynames:
    plt.figure()
    
#     if yname  == 'error':
#         df_ = df_.query('error < 150')
    sns.violinplot(df_, y=yname, x='pause_rel')

In [ ]:
df_.index

# summary many

In [ ]:
dfcc_all_ = dfcc_all.drop(labels=['level_0', 'index'], axis=1)
dfcc_all_ = dfcc_all_.reset_index().drop(labels=['index'], axis=1)
dfcc_all_

In [ ]:
numtrain = int(params['num_training'])
from behav_proc import qs_notspec_not_afterspec, spectrials
qs = qs_notspec_not_afterspec
df_ = dfcc_all.query(qs)

### Reaction time

In [ ]:
#same plot legend group by var

In [ ]:
#dfcc.columns

In [ ]:
df_all_multi_tsz['dist_rad_from_prevtgt2']

In [ ]:
[col for col in df_all_multi_tsz.columns if col.find('calc') >= 0]

In [ ]:
df_['dist_rad_from_prevtgt'].max()

In [ ]:
df__

In [ ]:
df_all_multi_tsz['early'] = df_all_multi_tsz['block_ind'] < max_bi / 2

In [ ]:
df_all_multi_tsz['early'].unique()

In [ ]:
gc.collect()

In [ ]:
df__ = df_all_multi_tsz.query(qs + ' and error_data_for_calc == @edfc')  
# there is much more point for dist_rad_from_prevtgt2 == 0
fg = sns.catplot(df__, kind='violin', y='time_lh', col='dist_rad_from_prevtgt2', row='early')
for ax in fg.axes.flatten():
    ax.set_ylim(-0.1,0.8)
    ax.axhline(ls=':',c='r')

In [ ]:
#dfes = dfcc.query(qs)# and err_sens >= -20 and err_sens <= 20')
fg = sns.relplot(df_,kind='line', y='time_lh', x='trial_index')#, x = 'error_data_for_calc')
# for ax in fg.axes.flatten():
#     ax.set_ylim(-10,10)

In [ ]:
7/4

In [ ]:
fg = sns.relplot(df_,kind='line', y='time_lh', x='trialwb')#, x = 'error_data_for_calc')

In [ ]:
df_.groupby(['subject', 'tgti_to_show', 'vis_feedback_type', 'trialwb']).size()

In [ ]:
ynames

In [ ]:
dfc_multi_tsz['error_data_for_calc'].unique()

In [ ]:
dfc_multi_tsz['trial_shift_size'].unique()

In [ ]:
len(dfes), len(dfc_multi_tsz)

In [ ]:
grp = dfes.groupby(['subject','error_data_for_calc'])
display(grp.size() )
grp = dfes.groupby(['error_data_for_calc'])
display(grp['err_sens'].mean())

In [ ]:
dfes = dfc_multi_tsz.query(qs + ' and trial_shift_size == 1')# and err_sens >= -20 and err_sens <= 20')
fg = sns.catplot(dfes, kind='violin',y='err_sens', x = 'error_data_for_calc')
for ax in fg.axes.flatten():
    ax.set_ylim(-3,3)
    ax.axhline(ls=':',c='r')

# Working with context similarity

In [ ]:
len(grp.groups[g])

In [ ]:
dfcc_all_.groupby(['subject','Nctx_app']).size().to_frame().iloc[:10]

In [ ]:
grp.size()

In [ ]:
dfcc_all.groupby(['subject','ctxid'])

In [ ]:
#grp.groups.keys()

In [ ]:
len(dfcc_tmp)

In [ ]:
dfcc_all

In [ ]:
int(np.nan)

In [ ]:
len(inds)

len(dfcc_all.loc[inds, pref + ln])

In [ ]:
len(pair_props[ln].to_numpy()), len(inds)

In [ ]:
from behav_proc import qs_notspec
qs = qs_notspec

In [ ]:
qs = qs_notspec_not_afterspec
print(qs)
df_ = dfcc_all.query(qs)

In [ ]:
dfcc.columns

In [ ]:
coln_ctx_closeness = ['prev_ctx_both_close',
       'prev_ctx_some_close', 'prev_ctx_tgt_close', 'prev_ctx_pert_close',
       'prev_ctx_pert_same', 'prev_ctx_tgt_same']

In [ ]:
np.sum(df_['prev_block_ind_diff'].isna())

In [ ]:
#sns.catplot?

In [ ]:
df_notsame = df_.query('prev_block_ind_diff > 0')
for c, cp in zip(['orange','blue'], coln_pairs):
    coln_error = cp[0]
    for coln in coln_ctx_closeness:
        plt.figure(figsize=(4,3))
        sns.catplot(df_notsame, y = coln_error, col=coln, kind='violin', height=3, color=c)
    


In [ ]:
len(ctx_pairs), len(dfcc)

In [ ]:
dfext_ = dfcc_all_.query('trial_index > @numtrain and trial_type != "error_clamp"')
fg = sns.catplot(dfext_, kind='violin', x='prev_trial_type', 
                 y='error_area2_signed_nn_scaled_ed', col = 'special_block_type')

fg = sns.catplot(dfext_, kind='violin', x='prev_trial_type', 
                 y='error_lh_ang', col = 'special_block_type')


# dfext_ = dfc_multi_tsz.query('trial_index > @numtrain and trial_type != "error_clamp"')
# fg = sns.catplot(dfext_, kind='violin', x='prev_trial_type', 
#                  y='error_area_signed_nn_scaled_ed', col = 'special_block_type')

In [ ]:
fg = sns.relplot(df_, kind='line', x='trialwb', y='error_area2_signed_nn_scaled_ed', 
                 col='vis_feedback_type', 
            row='tgti_to_show')
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',color='red')

In [ ]:
fg = sns.relplot(df_, kind='line', x='trialwb', y='error_lh_ang', 
                 col='vis_feedback_type', 
            row='tgti_to_show')
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',color='red')

In [ ]:
fg = sns.relplot(df_, kind='line', x='trialwb', y='error_area2_signed_nn_scaled_ed',
                 col='vis_feedback_type', 
            row='tgti_to_show', hue='nonhit', height=2)
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',color='red')

In [ ]:
fg = sns.relplot(df_, kind='line', x='trialwb', y='error_area2_signed_nn_scaled_ed', 
                 col='vis_feedback_type', 
            row='tgti_to_show', hue='Nctx_app', height=3)
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',color='red')

In [ ]:
qs_notspec

In [ ]:
[col for col in dfcc_all.columns if col.find('block') >= 0]

In [ ]:
dfcc_all['prev_ctx_some_close']

In [ ]:
len(dfcc_all)

In [ ]:
from behav_proc import genCtxPairLists,setContextSimilarityCols
all_ctx, cpls = genCtxPairLists(dfcc_all)
setContextSimilarityCols(dfcc_all, cpls)       

In [ ]:
(~dfcc_all['prev_ctx_some_close'].isnull()).sum()

In [ ]:
(~dfcc_all['prev_block_ind_diff'].isnull()).sum()

In [ ]:
df_notspec_notver = dfcc_all.query(qs_notspec + ' and vis_feedback_type != "veridical"'
                                  ' and trialwb <= 4')

In [ ]:
inds = 0,1,2,4

dfdrops = []
for colnp in coln_pairs:
    assert inds[0] == 0
    coln = colnp[0]
    
    def f(dfcur):
        #print(dfcur[coln])        
        dfs = []
        for ind in inds:
            dfs += [dfcur.query('trialwb == @ind')]
            
        #print(df1[coln])
        #print(df2[coln])
        vss = []
        for df in dfs[1:]:
            #vs = np.abs(dfs[0][coln].values - df[coln].values)
            vs = np.abs(dfs[0][coln].values) - np.abs(df[coln].values)
            vss += [vs]
        if len(vs) == 0:
            #display(dfcur)
            return None
        else:
            assert len(vs) ==1, len(vs)
            if dfs[0]['prev_ctx_pert_same'].values[0]:
                # Q: how to handle failed reaches?
                #print('Skipping due to same tgt')
                return None
            else:
                #v = (df1[coln].values - df3[coln].values)[0]
                return [vs[0] for vs in vss]
                #return [abs( vs[0] ), abs(v)]
    df_notspec_notver_difflearn = df_notspec_notver.\
        groupby(['subject','block_ind','Nctx_app', 'ctxid']).apply(f).reset_index()
    df_notspec_notver_difflearn= df_notspec_notver_difflearn.rename(columns={0:'absdrop'})
    df_notspec_notver_difflearn['coln'] = coln    
    dfdrops += [df_notspec_notver_difflearn]
    
dfdrop = pd.concat(dfdrops, ignore_index=1)
dfdrop

In [ ]:
for i,ind in enumerate(inds[1:] ):
    dfdrop[f'absdrop_{ind}'] = dfdrop['absdrop'].apply(lambda x: x[i] if x is not None else None)

In [ ]:
#dfdrop

In [ ]:
#df_notspec_notver_difflearn

In [ ]:
(~df_notspec_notver['prev_ctx_pert_same'].isnull()).sum()

In [ ]:
#sns.relplot?

In [ ]:
dfdrop

In [ ]:
# r = dfdrop.groupby(['coln', 'subject','ctxid']).agg({'Nctx_app': ['min', 'max'], 
#                                                 'absdrop_4': lambda x: x.max() - x.min()})
rs = []
for ind in inds[1:]:
    coln = f'absdrop_{ind}'
    # drop at last context apperence - drop at first one
    r = dfdrop.groupby(['coln', 'subject','ctxid']).apply(
    lambda x: np.abs( x.loc[x['Nctx_app'].idxmax(), coln] ) - np.abs (x.loc[x['Nctx_app'].idxmin(), coln] ) )
    r = r.to_frame().rename(columns={0:coln})#.reset_index()
    rs += [r]

dfdrop_last = pd.concat(rs, axis=1, ignore_index=0)

In [ ]:
dfdrop_last

In [ ]:
fg = sns.catplot(dfdrop, kind='violin', y='absdrop_4',  
             height=4, col='coln')
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',color='red')

fg = sns.catplot(dfdrop, kind='violin', x='ctxid', y='absdrop_4',  
             height=5, col='coln')
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',color='red')
    
fg = sns.catplot(dfdrop, kind='violin', y='absdrop_4',  
             height=4, row='coln', col='subject')
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',color='red')

In [ ]:
dfdrop.groupby('subject')

In [ ]:
dfdrop_ =dfdrop
fg = sns.relplot(dfdrop_, kind='line', x='Nctx_app', y='absdrop_4',  hue='ctxid',
            col='coln', height=3, errorbar='se')
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',color='red')
    #ax.set_ylabel('')
fg = sns.relplot(dfdrop_, kind='line', x='Nctx_app', y='absdrop_2',  hue='ctxid',
            col='coln', height=3, errorbar='se')
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',color='red')
    #ax.set_ylabel('')

In [ ]:
np.random.randint(0,3, size=20)

In [ ]:
df_notspec_notver = dfcc_all.query(qs_notspec + ' and vis_feedback_type != "veridical"'
                                  ' and trialwb <= 4 and Nctx_app in [1,7]')
for coln in ['error_area2_signed_nn_scaled_ed', 'error_lh_ang']:
    fg = sns.relplot(df_notspec_notver, kind='line', x='trialwb', y=coln, 
                     col='vis_feedback_type', hue='Nctx_app', 
                     row = 'subject', height=3, errorbar='se')
    for ax in fg.axes.flatten():
        ax.axhline(0,ls=':',color='red')
        ax.set_ylabel('')
        #ax.legend(loc='lower right')
    fg.fig.suptitle(coln)
    fg.fig.tight_layout()
    break

In [ ]:
df_os = df_.query('subject == @subjects[0] and vis_feedback_type == "rot20"')
fg = sns.relplot(df_os, kind='line', x='trialwb', y='error_area2_signed_nn_scaled_ed', 
                 col='vis_feedback_type', 
            row='tgti_to_show', hue='Nctx_app', height=3)
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',color='red')
    ax.set_ylabel('')

In [ ]:
dfc_multi_tsz.columns

### ES

In [ ]:
print( dfc_multi_tsz['trial_group_col_calc'].unique() )
print( dfc_multi_tsz['error_data_for_calc'].unique() )
print( dfc_multi_tsz.subject.unique() )

In [ ]:
#qs = ('trial_index > @numtrain and trial_type != "error_clamp"'
#                     ' and special_block_type != "error_clamp_sandwich" and prev_trial_type != "pause"')

In [ ]:
# TODO put to sameness info to dfcc before calc err sens 
for c, cp in zip(['orange','blue'], coln_pairs):
    coln_error = cp[0]
    edfs = cp[-1]
    print(cp)
    dfes = dfc_multi_tsz.query(qs + ' and trial_shift_size == 1 and error_data_for_calc == @edfc')
    dfes = dfes.copy() # otherwise the function below won't work since it modifies inplace
    setContextSimilarityCols(dfes)
    for coln in coln_ctx_closeness:
        plt.figure(figsize=(4,3))        
        fg = sns.catplot(dfes, y = 'err_sens', col=coln, kind='violin', height=3, color=c)
        for ax in fg.axes.flatten():
            ax.axhline(0,ls=':',color='red')
            ax.set_ylim(-20,20)
    gc.collect()

In [ ]:
plt.close('all')

In [ ]:
numtrain

In [ ]:
qs

In [ ]:
edfc

In [ ]:
for c, cp in zip(['orange','blue'], coln_pairs):
    #coln_error = cp[0]
    edfc = cp[-1]
    dfes = dfc_multi_tsz.query(qs + 
        ' and trial_shift_size == 1 and error_data_for_calc == @edfc')
    fg = sns.catplot(dfes, kind='violin', y='err_sens', height=3)
    for ax in fg.axes.flatten():
        ax.axhline(0,ls=':',color='red')
        ax.set_title(edfc)
        ax.set_ylim(-5,5)

In [ ]:
for c, cp in zip(['orange','blue'], coln_pairs):
    coln_error = cp[0]
    edfc = cp[-1]
    #print(edfs)
    dfes = dfc_multi_tsz.query(qs + ' and trial_shift_size == 1 and error_data_for_calc == @edfc')

    fg = sns.relplot(dfes, kind='line', x='trialwb', y='err_sens', hue='nonhit')
    for ax in fg.axes.flatten():
        ax.axhline(0,ls=':',color='red')
        ax.set_title(edfc)

In [ ]:
for edfc in ['signed_area2_nn_scaled_ed', 'lh_ang']:
    dfes = dfc_multi_tsz.query(qs + ' and trial_shift_size == 1 and error_data_for_calc == @edfc')

    fg = sns.relplot(dfes, kind='line', x='trialwb', y='err_sens',
                     col='vis_feedback_type', 
                row='tgti_to_show')#, hue='nonhit')
    for ax in fg.axes.flatten():
        ax.axhline(0,ls=':',color='red')
        plt.suptitle(edfc)
    plt.tight_layout()
    


In [ ]:
dfcc_all_['subject'].unique()

In [ ]:
df_startend['subject'].unique()

In [ ]:
df_startend

#### start end

In [ ]:
from behav_proc import getStartEndMultiSubj
dfes_startend = getStartEndMultiSubj(dfes)

In [ ]:
#block_bounds

In [ ]:
df_.query('subject == "2023-SE1-003" and block_ind == 12')

In [ ]:
#df_all_multi_tsz.query('subject == "2023-SE1-003" and block_ind == 12')

In [ ]:
dfcc.query('subject == "2023-SE1-003" and block_ind == 12')

In [ ]:
block_bounds[r == 1]

In [ ]:
r = block_bounds['block_end_trial_index'] - block_bounds['block_start_trial_index']
r.to_frame().describe()

In [ ]:
#dfes.groupby('subject').size()

In [ ]:
# from single imported
for c, cp in zip(['orange','blue'], coln_pairs):
    coln_error = cp[0]
    edfc = cp[-1]
    #print(edfs)
    dfes = dfc_multi_tsz.query(qs + ' and trial_shift_size == 1 and error_data_for_calc == @edfc')
    dfes_startend = getStartEndMultiSubj(dfes)

    for yname in ynames:
        plt.figure()
        ax = sns.violinplot(df_startend, y=yname, x='trial_type', hue='type', split=True)
        ax.axhline(0,ls=':', c='red')
        ax.set_ylim(-2,2)
        ax.set_title(edfc)

In [ ]:
for yname in ynames:
    plt.figure()
    q = 5e-3
    ymax = df_startend[yname].quantile(1-q)
    ymin = df_startend[yname].quantile(q)
    ax = sns.violinplot(df_startend.query('trial_type != "error_clamp"'), 
                   y=yname, x='trial_type', hue='type', split=True)
    ax.set_ylim(ymin,ymax)
    ax.axhline(0,ls=':',c='red')

In [ ]:
pause_inds = dfcpc_all.groupby(['subject','trial_index']).max('time').reset_index()

In [ ]:
pause_inds

In [ ]:
for subj in subjects:
    pause_inds.query('subject == @subj')
    c = dfcpc_all['subject'] == subj
    dfcpc_all.loc[c]

In [ ]:
#pause_inds = dfcpc_all['trial_index'].unique()

pause_inds = np.array(pause_inds)
dfccos['pause_rel'] = -100
dfccos.loc[dfccos['trial_index'].isin(pause_inds-1), 'pause_rel'] = -1
dfccos.loc[dfccos['trial_index'].isin(pause_inds+1), 'pause_rel'] = 1
# for pi in pause_inds:
#     piprev=  pi -1
#     pinext=  pi +1
#     df_ = dfccos.query('trial_index == @piprev')


In [ ]:
df_ = dfcc_all_.query('pause_rel > -10')
#df_ = df_.query('error < 150')

ynames = ['error_area_signed',  'error_distance', 'error']
for yname in ynames:
    plt.figure()
    
#     if yname  == 'error':
#         df_ = df_.query('error < 150')
    sns.violinplot(df_, y=yname, x='pause_rel')